<span style="color:Blue; font-family:Times New Roman; font-size:1.2em;">Import Required Packages</span>

In [1]:
### Data Processing Packages
import requests # web api 
import platform, os, sys # system, file,and directory 
import json, pprint # json 
import logging, timeit # run time 
import pandas as pd 
import numpy as np

### Datetime Processing Packages
from datetime import datetime # datetime 
import time, dateutil, pytz, calendar # UNIX Time
from tzlocal import get_localzone # timezone 

### Dashboard Packages
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

plotly.tools.set_credentials_file(username='jasonleetoronto2018', api_key='nj442HUT4BlVXOwawT1l')
init_notebook_mode(connected=True)

### Parallel Processing Packages
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed

<span style="color:Blue; font-family:Times New Roman; font-size:1.2em;">Check Current Working Directory</span>

In [2]:
print ("\033[2;37;40mCurrent file directory is in {} computer and file folder {}\n" .format(platform.uname()[1], os.getcwd()))
os.chdir("C:\\Users\\leejas\\Desktop\\Summer Project")
print (os.getcwd())

Current file directory is in YRK59206 computer and file folder C:\Users\leejas\Desktop\Summer Project

C:\Users\leejas\Desktop\Summer Project


<span style="color:Blue; font-family:Times New Roman; font-size:1.2em;">Read Smart Meter Information</span>

In [8]:
def trimAllColumns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trimStrings = lambda x: x.strip() if type(x) is str else x
    return df.applymap(trimStrings)

def read_meter_info():
    ### Create function for reading in smart meter ip address
    
    df_meter_info = pd.read_excel("meter_ip_list.xlsx")
    df_meter_info.head(5)

    for column in df_meter_info.columns:
        df_meter_info[column]=df_meter_info[column].astype(str)
        
    trimAllColumns(df_meter_info)    
    
    return df_meter_info

df_meter_info = read_meter_info()
df_meter_info.rename(columns={'IP Address':'IP_Address'}, inplace=True) # Remove space in column name
df_meter_info["Real_Time_Visualization"]=df_meter_info['IP_Address'].apply(lambda x: '<a href="http://'+x+'/istat.html">'+x+'</a>')

list_meter_name = list(df_meter_info["Meter"]) 
list_meter_ip = list(df_meter_info["IP_Address"]) 

table = ff.create_table(df_meter_info[["Meter","Real_Time_Visualization"]])
table.layout.width=400
py.iplot(table)


<span style="color:Blue; font-family:Times New Roman; font-size:1.2em;">Define Required Time Module</span>

In [3]:
def z3_to_posix(z3_date):
    
    ''' Changes epoch base z3_date(int)from 2010/01/01(Z3 Epoch time)\
    to 1970/01/01 (posix/UNIX Epoch time) 
    '''
    
    z3_epoch_base = "2010-01-01 00:00:00"    
    posix_epoch_base = "1970-01-01 00:00:00"
    
    time_format = "%Y-%m-%d %H:%M:%S"
    
    # Calculate time delta between z3_epoch_base_date & posix_epoch_base_date
    start_date = datetime.strptime(posix_epoch_base, time_format) # convert string into datetime
    end_date = datetime.strptime(z3_epoch_base, time_format) # convert string into datetime
    
    diff = end_date - start_date
    
    '''
    diff.days 
    diff.seconds//3600 
    diff.seconds % 3600/60
    diff.seconds % 3600 % 3600
    
    '''
    return z3_date + diff.total_seconds()

def posix_to_z3(posix_date):
    
    # Changes epoch base posix_date(int)from to 2010/01/01(Z3 Epoch time)

    
    z3_epoch_base = "2010-01-01 00:00:00"    
    posix_epoch_base = "1970-01-01 00:00:00"
    
    time_format = "%Y-%m-%d %H:%M:%S"
    
    # Calculate time delta between z3_epoch_base_date & posix_epoch_base_date
    start_date = datetime.strptime(posix_epoch_base, time_format) # convert string into datetime
    end_date = datetime.strptime(z3_epoch_base, time_format) # convert string into datetime
    
    diff = end_date - start_date
    
    return posix_date - diff.total_seconds()

def posix_to_local(utc_date):
    # Create a function for converting POSIX time to UTC time to local timezone time
        
    local_tz = get_localzone() # get local timezone
    utc_time = datetime.utcfromtimestamp(utc_date) # UTC Time
    local_time = utc_time.replace(tzinfo=pytz.utc).astimezone(local_tz) # Local Timezone Time
    
    return local_time

def local_to_posix(local_date):
    # Create a function for converting local time zone time to posix time
    
    local_tz = pytz.timezone(str(get_localzone())) # get local time zone & create pytz tz timezone object
    time_format = "%Y-%m-%d %H:%M:%S"
    
    local_date = datetime.strptime(local_date, time_format)
    local_date = local_tz.localize(local_date)
    local_date = local_date.astimezone(dateutil.tz.tzutc()).timetuple()
    
    return calendar.timegm(local_date)

def z3_to_local(z3_date):
    return posix_to_local(z3_to_posix(z3_date))

def local_to_z3(local_date):
    return posix_to_z3(local_to_posix(local_date))

def settlement_day_of_month(mydate, end=1):
    ### Create a function for calculating the last day of the month
    
    time_format = "%Y-%m-%d"
    mydate=datetime.strptime(mydate, time_format)
    
    if end==0:
        mydate=datetime(mydate.year,mydate.month,1)
    
    else:
        mydate=datetime(mydate.year,mydate.month,1)+dateutil.relativedelta.relativedelta(months=1,days=-1)
    
    return mydate

<span style="color:Blue; font-family:Times New Roman; font-size:1.2em;">Create ETL Module</span>

In [84]:
def fetch_datalog(meter_name, url):
    # Import API processing packages
    import requests # http package
    import json # json package
    
    # Fetch datalog smart meter data 
    try:
        url = requests.get("http://"+url+"/datalog.json?hdr=1")
        dict_datalog = json.loads(url.text)
        power_mult = dict_datalog["scale"][4]
        energy_mult = dict_datalog["scale"][6]
        column_names = dict_datalog["names"]
        list_power = dict_datalog["logdata"]
        df_power = pd.DataFrame(list_power,columns=column_names)
        
        df_power["Local Time"] = df_power["time"].apply(lambda x: datetime.strftime(z3_to_local(x),"%Y-%m-%d %H:%M:%S"))
        df_power["P(A+B+C)"] = df_power["P(A+B+C)"]*power_mult
        df_power["kWh(A+B+C)"] = df_power["WHr(A+B+C)"]*(energy_mult/1000)
        
        df_power.rename(columns={"time":"Z3 Datetime"}, inplace=True)
        df_datalog[meter_name]=df_power.sort_values(["Local Time"], ascending=[False])
        print "\033[2;34;40mSuccessfully fetched {} datalog from {} to {}\n" .format(meter_name,df_power["Local Time"].iloc[0], df_power["Local Time"].iloc[-1])
    
    except Exception as e:
        print "\033[2;31;43m{}\n".format(e) 
    

        Z3 Datetime           Local Time  P(A+B+C)  kWh(A+B+C)
0         263030820  2018-05-03 04:07:00  0.451803  481.421255
1         263030880  2018-05-03 04:08:00  0.301202  481.421255
2         263030940  2018-05-03 04:09:00  0.401603  481.421255
3         263031000  2018-05-03 04:10:00  0.351403  481.421255
4         263031060  2018-05-03 04:11:00  0.301202  481.421255
5         263031120  2018-05-03 04:12:00  0.376503  481.421255
6         263031180  2018-05-03 04:13:00  0.476903  481.421255
7         263031240  2018-05-03 04:14:00  0.301202  481.421255
8         263031300  2018-05-03 04:15:00  0.351403  481.421255
9         263031360  2018-05-03 04:16:00  0.376503  481.421255
10        263031420  2018-05-03 04:17:00  0.426703  481.421255
11        263031480  2018-05-03 04:18:00  0.251002  481.421255
12        263031540  2018-05-03 04:19:00  0.351403  481.421255
13        263031600  2018-05-03 04:20:00  0.577304  481.421255
14        263031660  2018-05-03 04:21:00  0.326302  481

In [30]:
def fetch_1min(meter_name, url):
    
    # Import API processing packages
    import requests # http package
    import json # json package
    
    # Fetch 1min smart meter data from start_date to end_date
    try:
        url = requests.get("http://"+url+"/sdata.json?m=f1t")
        dict_1min = json.loads(url.text)
        power_mult = dict_1min["pmul"]
        list_power = dict_1min["power"]
        df_power = pd.DataFrame(list_power)

        
        
        list_dt = []
        list_W = []
        list_kWh = [] 
    
        # Flatten nested list with z3_datetime and power_watt columns
        for _, row in df_power.iterrows():
            list_dt += range(row[0],row[0] + 60*(len(row[1])),60)
            list_W += row[1]

        list_W = pd.Series(list_W)*power_mult     
        list_kWh = list_W.cumsum()/(1000*60)

        data = {"date_time":list_dt,"W": list_W, "kWh":list_kWh}
        df_meter = pd.DataFrame(data)

        # Create meter_loaded and local_time columns
        df_meter["local_time"] = df_meter["date_time"].apply(lambda x: datetime.strftime(z3_to_local(x),"%Y-%m-%d %H:%M:%S"))

        df_meter = df_meter.loc[df_meter["local_time"] >= start_date]
        df_1min[meter_name] = df_meter.rename(columns={"date_time":"Z3 Datetime", "local_time":"Local Time", "W": "P(A+B+C)", "kWh":"kWh(A+B+C)"}, inplace = True)
        
        
        print "\033[2;36;40mSuccessfully fetched {} 1MIN data from {} to {}\n".format(meter_name, df_meter["Local Time"].iloc[0], df_meter["Local Time"].iloc[-1])

    except Exception as e:
        print "\033[2;31;43m{}\n".format(e) 
    

In [66]:
def fetch_1hour(meter_name, url):
    
    # Import API processing packages
    import requests # http package
    import json # json package
    
    # Create date subsetting api paramter
    t_str = "&t="+str(start_date) if len(str(start_date))!=0 else ""

    date_str = t_str

    # Fetch 1min smart meter data from start_date to end_date
    try:
        url = requests.get("http://"+url+"/sdata.json?m=f1h"+date_str)
        dict_1hour = json.loads(url.text)

        energy_mult = dict_1hour["emul"]
        
        list_power = dict_1hour["energy"]
        df_power = pd.DataFrame(list_power, columns=["date_time","Whr(A+B+C)"])
       
        df_power["Local Time"] = df_power["date_time"].apply(lambda x: datetime.strftime(z3_to_local(x),"%Y-%m-%d %H:%M:%S"))
        df_power["kWh(A+B+C)"] = df_power["Whr(A+B+C)"]*(energy_mult/1000)
        df_power["Loaded Meter"] = meter_name
        
        
        df_power.rename(columns={"date_time":"Z3 Datetime"}, inplace=True)
        
        df_power = df_power[["Z3 Datetime", "Local Time","kWh(A+B+C)"]]
        
        df_1hour[meter_name] = df_power.loc[df_power["Z3 Datetime"] >= start_date].sort_values(["Local Time"], ascending=[False])

        print "\033[2;36;40mSuccessfully stored {} 1HOUR data from {} to {}\n".format(meter_name, df_power["Local Time"].iloc[0], df_power["Local Time"].iloc[-1])    
        
    except Exception as e:
        print "\033[2;31;43m{}\n".format(e) 


<span style="color:Blue; font-family:Times New Roman; font-size:1.2em;">Data Extraction and Transform using Parallel Processing</span>

In [ ]:
### Datalog Data Extraction
df_datalog = {} # create temporary 1min dataframe dictionary

### Execute Parallel Processing Data Extraction using Cuncurrent.futures
with ThreadPoolExecutor(max_workers=3) as executor:
    args = ((meter_name, url)for meter_name, url in zip(list_meter_name, list_meter_ip))
    executor.map(lambda p: fetch_datalog(*p), args)
    
### Designate Starting and Ending Date for Data Extraction in %Y-%m-%d %H:%M:%S 
  
list_start=[]
list_end=[]

for key, value in df_datalog.items():
    list_start.append(value["Z3 Datetime"].iloc[-1])
    list_end.append(value["Z3 Datetime"].iloc[0])
    
start_date = pd.Series(list_start).min()

print "\033[2;34;40mSmart Meter Comparison is for the period of from {} to {}\n" .format(z3_to_local(start_date), z3_to_local(end_date))


("Connection broken: error(10054, 'An existing connection was forcibly closed by the remote host')", error(10054, 'An existing connection was forcibly closed by the remote host'))

       Z3 Datetime           Local Time   P(A+B+C)  kWh(A+B+C)
0        265400340  2018-05-30 14:19:00  18.259273  501.898665
1        265400400  2018-05-30 14:20:00  18.349037  501.902827
2        265400460  2018-05-30 14:21:00  19.831331  501.907250
3        265400520  2018-05-30 14:22:00  19.729138  501.911795
4        265400580  2018-05-30 14:23:00  19.693874  501.916302
5        265400640  2018-05-30 14:24:00  17.662958  501.920570
6        265400700  2018-05-30 14:25:00  19.567374  501.924894
7        265400760  2018-05-30 14:26:00  19.804772  501.929403
8        265400820  2018-05-30 14:27:00  19.436584  501.933907
9        265400880  2018-05-30 14:28:00  19.390008  501.938356
10       265400940  2018-05-30 14:29:00  19.289411  501.942777
11       265401000  2018-05-30 14:30:00  18.966698  501.947184


       Z3 Datetime           Local Time   P(A+B+C)  kWh(A+B+C)
0        263062800  2018-05-03 13:00:00  14.235390  943.359805
1        263062860  2018-05-03 13:01:00  13.410670  943.360077
2        263062920  2018-05-03 13:02:00  13.231383  943.360333
3        263062980  2018-05-03 13:03:00  27.359200  943.360602
4        263063040  2018-05-03 13:04:00  13.338955  943.360867
5        263063100  2018-05-03 13:05:00  12.872809  943.361127
6        263063160  2018-05-03 13:06:00  12.693522  943.361398
7        263063220  2018-05-03 13:07:00  13.518242  943.361655
8        263063280  2018-05-03 13:08:00  13.016238  943.361923
9        263063340  2018-05-03 13:09:00  12.908666  943.362183
10       263063400  2018-05-03 13:10:00  14.665679  943.362457
11       263063460  2018-05-03 13:11:00  13.267240  943.362717
12       263063520  2018-05-03 13:12:00  12.908666  943.362991
13       263063580  2018-05-03 13:13:00  25.745617  943.363254
14       263063640  2018-05-03 13:14:00  13.410670  943

        Z3 Datetime           Local Time  P(A+B+C)  kWh(A+B+C)
0         263034780  2018-05-03 05:13:00  1.649441  137.959192
1         263034840  2018-05-03 05:14:00  1.075722  137.959192
2         263034900  2018-05-03 05:15:00  0.860578  137.959192
3         263034960  2018-05-03 05:16:00  0.860578  137.959192
4         263035020  2018-05-03 05:17:00  0.932293  137.959192
5         263035080  2018-05-03 05:18:00  0.932293  137.959192
6         263035140  2018-05-03 05:19:00  1.147437  137.959192
7         263035200  2018-05-03 05:20:00  0.788863  137.959192
8         263035260  2018-05-03 05:21:00  1.434296  137.959192
9         263035320  2018-05-03 05:22:00  1.075722  137.959192
10        263035380  2018-05-03 05:23:00  1.362581  137.959192
11        263035440  2018-05-03 05:24:00  0.502004  137.959192
12        263035500  2018-05-03 05:25:00  1.004007  137.959192
13        263035560  2018-05-03 05:26:00  0.860578  137.959192
14        263035620  2018-05-03 05:27:00  1.219152  137

Successfully fetched Panel C4 Pumps datalog from 2018-07-05 09:11:00 to 2018-07-13 15:53:00

        Z3 Datetime           Local Time    P(A+B+C)    kWh(A+B+C)
0         262836840  2018-04-30 22:14:00  211.379405  63402.408624
1         262836900  2018-04-30 22:15:00  212.275840  63402.412686
2         262836960  2018-04-30 22:16:00  207.399233  63402.416748
3         262837020  2018-04-30 22:17:00  207.721950  63402.420807
4         262837080  2018-04-30 22:18:00  206.933087  63402.424863
5         262837140  2018-04-30 22:19:00  206.897230  63402.428915
6         262837200  2018-04-30 22:20:00  206.466941  63402.432971
7         262837260  2018-04-30 22:21:00  206.825515  63402.437015
8         262837320  2018-04-30 22:22:00  207.291661  63402.441064
9         262837380  2018-04-30 22:23:00  206.215939  63402.445114
10        262837440  2018-04-30 22:24:00  207.757807  63402.449164
11        262837500  2018-04-30 22:25:00  206.789657  63402.453214
12        262837560  2018-04-30 22:2

        Z3 Datetime           Local Time    P(A+B+C)    kWh(A+B+C)
0         262836840  2018-04-30 22:14:00  211.379405  63402.408624
1         262836900  2018-04-30 22:15:00  212.275840  63402.412686
2         262836960  2018-04-30 22:16:00  207.399233  63402.416748
3         262837020  2018-04-30 22:17:00  207.721950  63402.420807
4         262837080  2018-04-30 22:18:00  206.933087  63402.424863
5         262837140  2018-04-30 22:19:00  206.897230  63402.428915
6         262837200  2018-04-30 22:20:00  206.466941  63402.432971
7         262837260  2018-04-30 22:21:00  206.825515  63402.437015
8         262837320  2018-04-30 22:22:00  207.291661  63402.441064
9         262837380  2018-04-30 22:23:00  206.215939  63402.445114
10        262837440  2018-04-30 22:24:00  207.757807  63402.449164
11        262837500  2018-04-30 22:25:00  206.789657  63402.453214
12        262837560  2018-04-30 22:26:00  207.721950  63402.457269
13        262837620  2018-04-30 22:27:00  207.255804  63402.46

In [61]:
### 1MIN Data Extraction
df_1min = {} # create temporary 1min dataframe dictionary

### Execute Parallel Processing Data Extraction using 
with ThreadPoolExecutor(max_workers=3) as executor:
    args = ((meter_name, url)for meter_name, url in zip(list_meter_name, list_meter_ip))
    executor.map(lambda p: fetch_1min(*p), args)

KeyboardInterrupt: 

In [67]:
### 1HOUR Data Extraction
df_1hour = {} # create temporary 1hour dataframe dictionary

### Execute Parallel Processing Data Extraction using 
with ThreadPoolExecutor(max_workers=3) as executor:
    args = ((meter_name, url)for meter_name, url in zip(list_meter_name, list_meter_ip))
    executor.map(lambda p: fetch_1hour(*p), args)

Successfully stored TX1 Panel A 1HOUR data from 2018-07-13 15:25:14 to 2018-04-14 16:00:00

Successfully stored TX3 Panel B 1HOUR data from 2018-07-13 15:25:14 to 2018-04-14 16:00:00

Successfully stored Incoming Electrical Service 1HOUR data from 2018-07-13 15:25:14 to 2018-04-14 16:00:00

Successfully stored Vehicle Charger 2 1HOUR data from 2018-07-13 15:25:18 to 2018-04-14 16:00:00

Successfully stored Vehicle Charger 1 1HOUR data from 2018-07-13 15:25:17 to 2018-04-14 16:00:00

Successfully stored Exterior Lighting 1HOUR data from 2018-07-13 15:25:18 to 2018-04-14 16:00:00

Successfully stored Panel C3 DHW 1HOUR data from 2018-07-13 15:25:22 to 2018-04-14 16:00:00

Successfully stored Panel C1 Lighting 1HOUR data from 2018-07-13 15:25:22 to 2018-04-14 16:00:00

Successfully stored Panel C2 Lighting 1HOUR data from 2018-07-13 15:25:23 to 2018-04-14 16:00:00

Successfully stored Panel C4 Pumps 1HOUR data from 2018-07-13 15:25:26 to 2018-04-14 16:00:00

Successfully stored Natures Cl

<span style="color:Blue; font-family:Times New Roman; font-size:1.2em;">Datalog vs 1MIN</span>

In [ ]:
### Create Available Data Records, Dat

df_temp = {}

for meter_name in list_meter_name:
    beg_date = df_datalog["Z3 Datetime"].iloc[0]
    end_date = df_1min["Z3 Datetime"].iloc[-1]
    df_temp[meter_name]=[df_datalog["Local Time"].iloc[0] , df_1min["Local Time"].iloc[-1] ,df_datalog[[df_datalog["Z3 Datetime"]<=end_date]].shape[0], df_1min[df_1min["Z3 Datetime"]>=beg_date].shape[0], (end_date- beg_date)/60]
    
df_datalog_1min = pd.DataFrame.from_dict(df_temp, orient='index').rename(columns={0:"Begin Date", 1:"End Date", 2:"Datalog Minutes", 3:"1MIN Minutes", 4:"Available Minutes"})    

table = ff.create_table(df_datalog_1min)
table.layout.width=800
py.iplot(table)

<span style="color:Blue; font-family:Times New Roman; font-size:1.2em;">Datalog vs 1HOUR</span>

In [ ]:
df_temp = {}

for meter_name in list_meter_name:
    df_dl = df_datalog[meter_name]
    Filter=pd.to_datetime(df_dl["Local Time"], format="%Y-%m-%d %H:%M:%S").minute==0
    df_dl = df_dl[Filter]
    
    df_hr = df_1hour[meter_name]
    
    beg_date = df_dl["Z3 Datetime"].iloc[0]
    end_date = df_hr["Z3 Datetime"].iloc[-1]
    

    df_temp[meter_name]=[df_hr["Local Time"].iloc[0], df_dl["Local Time"].iloc[-1] , df_dl[meter_name].shape[0], df_hr[meter_name].shape[0], (end_date - beg_date)/(60*60)]

df_datalog_1hour = pd.DataFrame.from_dict(df_temp, orient='index').rename(columns={0:"Begin Date", 1:"End Date", 2:"Datalog Hours", 3:"1HOUR HOURS", 4:"Available Hours"})    

table = ff.create_table(df_datalog_1hour)
table.layout.width=800
py.iplot(table)    
    
    
    